# FlowDB Dataset Building 2.0 Play Notebook

(I) Get the data from the USGS website for a specified time period and site number. Here we will use site 01010500

In [ ]:
# site_id 01010500
# site_id 02378500 with precip
# site_id
import pandas as pd
from datetime import datetime
from typing import Tuple, Dict
import requests

def make_usgs_data(start_date:datetime, end_date:datetime, site_number:str):
    """
    Function that scrapes data from gages from a specified start_time THROUGH
    a specified end_time. Returns hourly df of river flow data. For instance,

    ..
    from datetime import datetime
    df = make_usgs_data(datetime(2020, 5, 1), datetime(2021, 5, 1) "01010500")
    df[-24:] # would return time stamps of 5/1
    ..
    """
    #//waterservices.usgs.gov/nwis/iv/?format=rdb,1.0&sites={}&startDT={}&endDT={}&parameterCd=00060,00065,00045&siteStatus=all
    base_url = "http://waterservices.usgs.gov/nwis/iv/?format=rdb,1.0&sites={}&startDT={}&endDT={}&parameterCd=00060,00065,00045&siteStatus=all"
    full_url = base_url.format(site_number, start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"))
    print("Getting request from USGS")
    print(full_url)
    r = requests.get(full_url)
    with open(site_number + ".txt", "w") as f:
        f.write(r.text)
    print("Request finished")
    response_data = process_response_text(site_number + ".txt")
    create_csv(response_data[0], response_data[1], site_number)
    return pd.read_csv(site_number + "_flow_data.csv")

def column_renamer(x):
  code_converter_1 = {"00060": "cfs", "00065":"height", "00045": "precip_usgs"}
  split_x = x.split("_")
  if len(split_x) > 1:
    if split_x[1] in code_converter_1 and "cd" not in x:
      return code_converter_1[split_x[1]]
  return x

def rename_cols(df) -> pd.DataFrame:

  df.columns = df.columns.map(column_renamer)
  return df


def process_response_text(file_name:str)->Tuple[str, Dict]:
    extractive_params = {}
    with open(file_name, "r") as f:
        lines = f.readlines()
        i = 0
        params = False
        while "#" in lines[i]:
            # TODO figure out getting height and discharge code efficently
            the_split_line = lines[i].split()[1:]
            if params:
                print(the_split_line)
                if len(the_split_line)<2:
                    params = False
                else:
                    extractive_params[the_split_line[0]+"_"+the_split_line[1]] = df_label(the_split_line[2])
            if len(the_split_line)>2:
                if the_split_line[0]== "TS":
                    params = True
            i+=1
        with open(file_name.split(".")[0] + "data.tsv", "w") as t:
            t.write("".join(lines[i:]))
        return file_name.split(".")[0] + "data.tsv", extractive_params

def df_label(usgs_text:str)->str:
    usgs_text = usgs_text.replace(",","")
    if usgs_text == "Discharge":
        return "cfs"
    elif usgs_text=="Gage":
        return "height"
    else:
        return usgs_text

def create_csv(file_path:str, params_names:dict, site_number:str):
    """
    Function that creates the final version of the CSV file
    Assigns
    """
    print(params_names)
    df = pd.read_csv(file_path, sep="\t")
    for key, value in params_names.items():
        df[value] = df[key]
    df.to_csv(site_number + "_flow_data.csv")


In [ ]:
df = make_usgs_data(datetime(2023, 7, 27), datetime(2023, 7, 27), "01010500")

Getting request from USGS
http://waterservices.usgs.gov/nwis/iv/?format=rdb,1.0&sites=01010500&startDT=2023-07-27&endDT=2023-07-27&parameterCd=00060,00065,00045&siteStatus=all
Request finished
{}


In [ ]:
df

,Unnamed: 0,agency_cd,site_no,datetime,tz_cd,cfs,65342_00060_cd,height,65341_00065_cd
0,0,5s,15s,20d,6s,14n,10s,14n,10s
1,1,USGS,01010500,2023-07-27 00:00,EDT,2300,P,5.10,P
2,2,USGS,01010500,2023-07-27 00:15,EDT,2300,P,5.10,P
3,3,USGS,01010500,2023-07-27 00:30,EDT,2300,P,5.10,P
4,4,USGS,01010500,2023-07-27 00:45,EDT,2280,P,5.09,P
...,...,...,...,...,...,...,...,...,...
92,92,USGS,01010500,2023-07-27 22:45,EDT,2040,P,4.97,P
93,93,USGS,01010500,2023-07-27 23:00,EDT,2040,P,4.97,P
94,94,USGS,01010500,2023-07-27 23:15,EDT,2040,P,4.97,P
95,95,USGS,01010500,2023-07-27 23:30,EDT,2040,P,4.97,P


In [ ]:
rename_cols(df)

,Unnamed: 0,agency_cd,site_no,datetime,tz_cd,cfs,65342_00060_cd,height,65341_00065_cd
0,0,5s,15s,20d,6s,14n,10s,14n,10s
1,1,USGS,01010500,2023-07-27 00:00,EDT,2300,P,5.10,P
2,2,USGS,01010500,2023-07-27 00:15,EDT,2300,P,5.10,P
3,3,USGS,01010500,2023-07-27 00:30,EDT,2300,P,5.10,P
4,4,USGS,01010500,2023-07-27 00:45,EDT,2280,P,5.09,P
...,...,...,...,...,...,...,...,...,...
92,92,USGS,01010500,2023-07-27 22:45,EDT,2040,P,4.97,P
93,93,USGS,01010500,2023-07-27 23:00,EDT,2040,P,4.97,P
94,94,USGS,01010500,2023-07-27 23:15,EDT,2040,P,4.97,P
95,95,USGS,01010500,2023-07-27 23:30,EDT,2040,P,4.97,P


In [ ]:
df.tail(20)

,Unnamed: 0,agency_cd,site_no,datetime,tz_cd,cfs,65342_00060_cd,height,65341_00065_cd
77,77,USGS,01010500,2023-07-27 19:00,EDT,2080,P,4.99,P
78,78,USGS,01010500,2023-07-27 19:15,EDT,2080,P,4.99,P
79,79,USGS,01010500,2023-07-27 19:30,EDT,2060,P,4.98,P
80,80,USGS,01010500,2023-07-27 19:45,EDT,2060,P,4.98,P
81,81,USGS,01010500,2023-07-27 20:00,EDT,2060,P,4.98,P
82,82,USGS,01010500,2023-07-27 20:15,EDT,2060,P,4.98,P
83,83,USGS,01010500,2023-07-27 20:30,EDT,2060,P,4.98,P
84,84,USGS,01010500,2023-07-27 20:45,EDT,2060,P,4.98,P
85,85,USGS,01010500,2023-07-27 21:00,EDT,2060,P,4.98,P
86,86,USGS,01010500,2023-07-27 21:15,EDT,2040,P,4.97,P


In [ ]:
df["height"].isna().sum()

0

In [ ]:
df["cfs"].isna().sum()

0

We will now process the data-frame into the proper format.

In [ ]:
import pytz
from typing import Tuple

timezone_map = {"EST": "America/New_York", "EDT":"America/New_York", "CST":"America/Chicago", "CDT":"America/Chicago", "MDT":"America/Denver", "MST":"America/Denver",
                "PST": "America/Los_Angeles", "PDT": "America/Los_Angeles"}

def process_intermediate_csv(df:pd.DataFrame) -> (pd.DataFrame, int, int, int):
  """
  Converts local time to UTC time, counts NaN values, gets max/min flows
  """
  # Remove garbage first row
  # TODO check if more rows are garabage
  df = df.iloc[1:]
  time_zone = df["tz_cd"].iloc[0]
  time_zone = timezone_map[time_zone]
  old_timezone = pytz.timezone(time_zone)
  new_timezone = pytz.timezone("UTC")
  # This assumes timezones are consistent throughout the USGS stream (this should be true)
  df["datetime"] = df["datetime"].map(lambda x: old_timezone.localize(datetime.strptime(x, "%Y-%m-%d %H:%M")).astimezone(new_timezone))
  df["cfs"] = pd.to_numeric(df['cfs'], errors='coerce')
  max_flow = df["cfs"].max()
  min_flow = df["cfs"].min()
  # doesn't do anything with count of na?
  count_nan = len(df["cfs"]) - df["cfs"].count()
  return df[df.datetime.dt.minute==0], max_flow, min_flow



In [ ]:
flow_df = process_intermediate_csv(df)

<ipython-input-38-ace7773ccafc>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["datetime"] = df["datetime"].map(lambda x: old_timezone.localize(datetime.strptime(x, "%Y-%m-%d %H:%M")).astimezone(new_timezone))
<ipython-input-38-ace7773ccafc>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cfs"] = pd.to_numeric(df['cfs'], errors='coerce')


We will now download the meta-data for this station in order to continue to getting the precipitation data from ASOS.

In [ ]:
!gsutil cp gs://flow_datasets/01010500.json .

Copying gs://flow_datasets/01010500.json...
/ [1 files][ 14.4 KiB/ 14.4 KiB]                                                
Operation completed over 1 objects/14.4 KiB.                                     


In [ ]:
import json
with open("01010500.json") as m:
  meta_data = json.load(m)

FileNotFoundError: ignored

We will now look at the format of the meta-data. The meta-data should always contain the following

```
"id": string
"latitude": float
"longtitude": float
"name": string
"state": string
"timeZoneInfo": Dict
```
`name`: Is a description of the gage. Usually includes river name and closest city.

`timeZoneDict` is a dictionary contained "zoneOffset" and "zoneAbbreviation"

In [ ]:
meta_data

{'id': '01010500',
 'latitude': 47.11305556,
 'logitude': -69.0880556,
 'name': 'St. John River at Dickey, Maine',
 'state': 'ME',
 'timeZoneInfo': "{'zoneOffset': '-05:00', 'zoneAbbreviation': 'EST'}",
 'elevation': {'results': [{'elevation': 181.9550170898438,
    'location': {'lat': 47.11305556, 'lng': -69.0880556},
    'resolution': 19.08790397644043}],
  'status': 'OK'},
 'soil_data': {'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [-69.0880556, 47.11305556]},
  'properties': {'layers': [{'name': 'bdod',
     'unit_measure': {'d_factor': 100,
      'mapped_units': 'cg/cm³',
      'target_units': 'kg/dm³',
      'uncertainty_unit': ''},
     'depths': [{'range': {'top_depth': 0,
        'bottom_depth': 5,
        'unit_depth': 'cm'},
       'label': '0-5cm',
       'values': {'Q0.05': None,
        'Q0.5': None,
        'Q0.95': None,
        'mean': None,
        'uncertainty': None}},
      {'range': {'top_depth': 5, 'bottom_depth': 15, 'unit_depth': 'cm'},
   

In [ ]:
meta_data["stations"]

[{'station_id': 'FVE',
  'station_json': '{"Unnamed: 0":2014,"Precip":null,"Time":null,"begints":"4\\/1\\/88 9:00","elev":301.0,"iem_network":null,"iem_network\\\\":"ME_ASOS\\\\","lat":47.2855,"lon":-68.31272,"station_name":"FRENCHVILLE","stid":"FVE"}',
  'distance': 61.63594291045976,
  'missing_precip': 512679,
  'missing_temp': 397406},
 {'station_id': '40B',
  'station_json': '{"Unnamed: 0":1217,"Precip":null,"Time":null,"begints":"3\\/1\\/85 6:00","elev":314.0,"iem_network":null,"iem_network\\\\":"ME_ASOS\\\\","lat":46.609,"lon":-69.531,"station_name":"CLAYTON LAKE","stid":"40B"}',
  'distance': 65.387980421327,
  'missing_precip': 79082,
  'missing_temp': 61},
 {'station_id': 'CAR',
  'station_json': '{"Unnamed: 0":992,"Precip":null,"Time":null,"begints":"12\\/31\\/47 23:00","elev":190.0,"iem_network":null,"iem_network\\\\":"ME_ASOS\\\\","lat":46.87,"lon":-68.02,"station_name":"CARIBOU MUNICIPAL","stid":"CAR"}',
  'distance': 85.3971826432755,
  'missing_precip': 505812,
  'missi

Now lets get the weather data for these station ASOS.

In [ ]:
from datetime import datetime, timedelta
def get_asos_data_from_url(station_id, base_url, start_time, end_time, station={}, stations_explored={}):
        """
        end_time: End date should always be plus one of the date scraped by the USGS function.
        """
        # TODO change URL to get non ASOS gages
        if "saved_complete" not in stations_explored:
          stations_explored["saved_complete"] = {}
        response = requests.get(base_url.format(station_id, start_time.year, start_time.month, start_time.day, end_time.year, end_time.month, end_time.day))
        with open("temp_weather_data.csv", "w+") as f:
          f.write(response.text)
        df, missing_precip, missing_temp = process_asos_csv("temp_weather_data.csv")
        station["missing_precip"] = missing_precip
        station["missing_temp"] = missing_temp
        stations_explored["saved_complete"][station_id] = station
        df.to_csv(str(station_id)+".csv")
        # Caching code
        # name = str(station["station_id"])+".csv"
        # upload_file("predict_cfs",  "asos_new/" + name, name, client)
        # station_meta_dict[station["station_id"]] = station
        # stations_list.append(station)

def process_asos_csv(path:str):
    df = pd.read_csv(path)
    print(df)
    missing_precip = df['p01m'][df['p01m']=='M'].count()
    missing_temp = df['tmpf'][df['tmpf']=='M'].count()
    df['hour_updated'] = df['valid'].map(format_dt)
    df['tmpf'] = pd.to_numeric(df['tmpf'], errors='coerce')
    df['dwpf'] = pd.to_numeric(df['dwpf'], errors='coerce')
    df['p01m'] = pd.to_numeric(df['p01m'], errors='coerce')
    print(df)
    # Replace mising values with an average of the two closest values
    # Since stations record at different intervals this could
    # actually cause an overestimation of precip. Instead replace with 0
    #df['p01m']=(df['p01m'].fillna(method='ffill') + df['p01m'].fillna(method='bfill'))/2
    #df['p01m'] = df['p01m'].fillna(0)
    #df['tmpf']=(df['tmpf'].fillna(method='ffill') + df['tmpf'].fillna(method='bfill'))/2
    df = df.groupby(by=['hour_updated'], as_index=False).agg({'p01m': 'sum', 'valid': 'first', 'tmpf': 'mean', 'dwpf':'mean', 'ice_accretion_1hr':'first', 'mslp':'first', 'drct':'first'})
    return df, int(missing_precip), int(missing_temp)

def format_dt(date_time_str:str) -> datetime:
  proper_datetime = datetime.strptime(date_time_str, "%Y-%m-%d %H:%M")
  if proper_datetime.minute != 0:
    proper_datetime = proper_datetime + timedelta(hours=1)
    proper_datetime = proper_datetime.replace(minute=0)
  return proper_datetime


In [ ]:
base_url = "https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station={}&data=tmpf&data=p01m&data=dwpf&data=relh&data=feel&data=drct&data=sped&data=mslp&data=ice_accretion_1hr&year1={}&month1={}&day1={}&year2={}&month2={}&day2={}&tz=Etc%2FUTC&format=onlycomma&latlon=no&missing=M&trace=T&direct=no&report_type=1&report_type=2"
get_asos_data_from_url("FVE", base_url, datetime(2023, 7, 27), datetime(2023, 7, 29))

    station             valid   tmpf  p01m   dwpf   relh   feel    drct  sped  \
0       FVE  2023-07-27 00:00      M     M      M      M      M  170.00  2.30   
1       FVE  2023-07-27 00:05      M     M      M      M      M  170.00  2.30   
2       FVE  2023-07-27 00:10      M     M      M      M      M  180.00  2.30   
3       FVE  2023-07-27 00:15      M     M      M      M      M  170.00  2.30   
4       FVE  2023-07-27 00:20      M     M      M      M      M  160.00  2.30   
..      ...               ...    ...   ...    ...    ...    ...     ...   ...   
628     FVE  2023-07-28 23:40      M     M      M      M      M  180.00  4.60   
629     FVE  2023-07-28 23:45      M     M      M      M      M  180.00  4.60   
630     FVE  2023-07-28 23:50      M     M      M      M      M  190.00  4.60   
631     FVE  2023-07-28 23:53  71.00  0.00  65.00  81.34  71.00  190.00  4.60   
632     FVE  2023-07-28 23:55      M     M      M      M      M  180.00  2.30   

        mslp ice_accretion_

In [ ]:
fve_df, missing_precip, missing_temp = process_asos_csv("FVE.csv")

    Unnamed: 0         hour_updated   p01m             valid       tmpf  \
0            0  2023-07-27 00:00:00   0.00  2023-07-27 00:00        NaN   
1            1  2023-07-27 01:00:00   0.00  2023-07-27 00:05  71.000000   
2            2  2023-07-27 02:00:00   0.00  2023-07-27 01:05  70.000000   
3            3  2023-07-27 03:00:00   0.00  2023-07-27 02:05  69.000000   
4            4  2023-07-27 04:00:00   0.00  2023-07-27 03:05  69.000000   
5            5  2023-07-27 05:00:00   0.00  2023-07-27 04:05        NaN   
6            6  2023-07-27 06:00:00   0.00  2023-07-27 05:45  68.000000   
7            7  2023-07-27 07:00:00   5.84  2023-07-27 06:05  67.000000   
8            8  2023-07-27 08:00:00   4.57  2023-07-27 07:05  67.000000   
9            9  2023-07-27 09:00:00   2.75  2023-07-27 08:05  66.000000   
10          10  2023-07-27 10:00:00   2.50  2023-07-27 09:05  66.000000   
11          11  2023-07-27 11:00:00   0.00  2023-07-27 10:05  66.000000   
12          12  2023-07-2

In [ ]:
fve_df

,hour_updated,p01m,valid,tmpf,dwpf,ice_accretion_1hr,mslp,drct
0,2023-07-27 00:00:00,0.00,2023-07-27 00:00,NaN,NaN,M,M,170.00
1,2023-07-27 01:00:00,0.00,2023-07-27 00:05,71.000000,62.000000,M,M,170.00
2,2023-07-27 02:00:00,0.00,2023-07-27 01:05,70.000000,62.000000,M,M,190.00
3,2023-07-27 03:00:00,0.00,2023-07-27 02:05,69.000000,62.000000,M,M,180.00
4,2023-07-27 04:00:00,0.00,2023-07-27 03:05,69.000000,62.000000,M,M,200.00
5,2023-07-27 05:00:00,0.00,2023-07-27 04:05,NaN,NaN,M,M,180.00
6,2023-07-27 06:00:00,0.00,2023-07-27 05:45,68.000000,62.000000,M,M,M
7,2023-07-27 07:00:00,5.84,2023-07-27 06:05,67.000000,64.000000,M,M,210.00
8,2023-07-27 08:00:00,4.57,2023-07-27 07:05,67.000000,64.000000,M,M,230.00
9,2023-07-27 09:00:00,2.75,2023-07-27 08:05,66.000000,65.000000,M,M,220.00


In [ ]:
def combine_data(flow_df, precip_df):
  tz = pytz.timezone("UTC")
  precip_df['hour_updated'] = precip_df['hour_updated'].map(lambda x: x.tz_localize("UTC"))
  joined_df = precip_df.merge(flow_df, left_on='hour_updated', right_on='datetime', how='inner')
  nan_precip = sum(pd.isnull(joined_df['p01m']))
  nan_flow = sum(pd.isnull(joined_df['cfs']))
  return joined_df, nan_flow, nan_precip
combined_df, missing_flow, missing_precip = combine_data(flow_df[0], fve_df)

In [ ]:
flow_df[0]

,Unnamed: 0,agency_cd,site_no,datetime,tz_cd,cfs,65342_00060_cd,height,65341_00065_cd
1,1,USGS,01010500,2023-07-27 04:00:00+00:00,EDT,2300,P,5.10,P
5,5,USGS,01010500,2023-07-27 05:00:00+00:00,EDT,2280,P,5.09,P
9,9,USGS,01010500,2023-07-27 06:00:00+00:00,EDT,2300,P,5.10,P
13,13,USGS,01010500,2023-07-27 07:00:00+00:00,EDT,2300,P,5.10,P
17,17,USGS,01010500,2023-07-27 08:00:00+00:00,EDT,2280,P,5.09,P
21,21,USGS,01010500,2023-07-27 09:00:00+00:00,EDT,2260,P,5.08,P
25,25,USGS,01010500,2023-07-27 10:00:00+00:00,EDT,2240,P,5.07,P
29,29,USGS,01010500,2023-07-27 11:00:00+00:00,EDT,2220,P,5.06,P
33,33,USGS,01010500,2023-07-27 12:00:00+00:00,EDT,2220,P,5.06,P
37,37,USGS,01010500,2023-07-27 13:00:00+00:00,EDT,2200,P,5.05,P


## Exploring the combined DataFrame

Now we will explore the df for inconsistencies in the data and analyze those:

In [ ]:
combined_df

,hour_updated,p01m,valid,tmpf,dwpf,ice_accretion_1hr,mslp,drct,Unnamed: 0,agency_cd,site_no,datetime,tz_cd,cfs,65342_00060_cd,height,65341_00065_cd
0,2023-07-27 04:00:00+00:00,0.00,2023-07-27 03:05,69.0,62.0,M,M,200.00,1,USGS,01010500,2023-07-27 04:00:00+00:00,EDT,2300,P,5.10,P
1,2023-07-27 05:00:00+00:00,0.00,2023-07-27 04:05,NaN,NaN,M,M,180.00,5,USGS,01010500,2023-07-27 05:00:00+00:00,EDT,2280,P,5.09,P
2,2023-07-27 06:00:00+00:00,0.00,2023-07-27 05:45,68.0,62.0,M,M,M,9,USGS,01010500,2023-07-27 06:00:00+00:00,EDT,2300,P,5.10,P
3,2023-07-27 07:00:00+00:00,5.84,2023-07-27 06:05,67.0,64.0,M,M,210.00,13,USGS,01010500,2023-07-27 07:00:00+00:00,EDT,2300,P,5.10,P
4,2023-07-27 08:00:00+00:00,4.57,2023-07-27 07:05,67.0,64.0,M,M,230.00,17,USGS,01010500,2023-07-27 08:00:00+00:00,EDT,2280,P,5.09,P
5,2023-07-27 09:00:00+00:00,2.75,2023-07-27 08:05,66.0,65.0,M,M,220.00,21,USGS,01010500,2023-07-27 09:00:00+00:00,EDT,2260,P,5.08,P
6,2023-07-27 10:00:00+00:00,2.50,2023-07-27 09:05,66.0,64.0,M,M,200.00,25,USGS,01010500,2023-07-27 10:00:00+00:00,EDT,2240,P,5.07,P
7,2023-07-27 11:00:00+00:00,0.00,2023-07-27 10:05,66.0,64.0,M,M,210.00,29,USGS,01010500,2023-07-27 11:00:00+00:00,EDT,2220,P,5.06,P
8,2023-07-27 12:00:00+00:00,0.00,2023-07-27 11:05,67.0,64.0,M,M,240.00,33,USGS,01010500,2023-07-27 12:00:00+00:00,EDT,2220,P,5.06,P
9,2023-07-27 13:00:00+00:00,0.00,2023-07-27 12:05,68.0,64.0,M,M,250.00,37,USGS,01010500,2023-07-27 13:00:00+00:00,EDT,2200,P,5.05,P


In [ ]:
len(combined_df)

24

In [ ]:
fve_df

,hour_updated,p01m,valid,tmpf,dwpf,ice_accretion_1hr,mslp,drct
0,2023-07-27 00:00:00+00:00,0.00,2023-07-27 00:00,NaN,NaN,M,M,170.00
1,2023-07-27 01:00:00+00:00,0.00,2023-07-27 00:05,71.0,62.0,M,M,170.00
2,2023-07-27 02:00:00+00:00,0.00,2023-07-27 01:05,70.0,62.0,M,M,190.00
3,2023-07-27 03:00:00+00:00,0.00,2023-07-27 02:05,69.0,62.0,M,M,180.00
4,2023-07-27 04:00:00+00:00,0.00,2023-07-27 03:05,69.0,62.0,M,M,200.00
5,2023-07-27 05:00:00+00:00,0.00,2023-07-27 04:05,NaN,NaN,M,M,180.00
6,2023-07-27 06:00:00+00:00,0.00,2023-07-27 05:45,68.0,62.0,M,M,M
7,2023-07-27 07:00:00+00:00,5.84,2023-07-27 06:05,67.0,64.0,M,M,210.00
8,2023-07-27 08:00:00+00:00,4.57,2023-07-27 07:05,67.0,64.0,M,M,230.00
9,2023-07-27 09:00:00+00:00,2.75,2023-07-27 08:05,66.0,65.0,M,M,220.00
